In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

258080768/258076736 [==============================] - 38s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [ ]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 40s - loss: 2.3949 - auc: 0.4964 - accuracy: 0.4963 - val_loss: 0.7282 - val_auc: 0.5000 - val_accuracy: 0.1719
Epoch 2/30
40/40 - 34s - loss: 0.7218 - auc: 0.5094 - accuracy: 0.5116 - val_loss: 0.7842 - val_auc: 0.5356 - val_accuracy: 0.1719
Epoch 3/30
40/40 - 34s - loss: 0.7436 - auc: 0.4996 - accuracy: 0.4982 - val_loss: 0.9929 - val_auc: 0.5850 - val_accuracy: 0.1719
Epoch 4/30
40/40 - 33s - loss: 0.7494 - auc: 0.5176 - accuracy: 0.5163 - val_loss: 0.5188 - val_auc: 0.5619 - val_accuracy: 0.8281
Epoch 5/30
40/40 - 33s - loss: 0.7398 - auc: 0.5089 - accuracy: 0.5120 - val_loss: 0.9231 - val_auc: 0.5534 - val_accuracy: 0.1693
Epoch 6/30
40/40 - 33s - loss: 0.7881 - auc: 0.4978 - accuracy: 0.5041 - val_loss: 0.6919 - val_auc: 0.5000 - val_accuracy: 0.8047
Epoch 7/30
40/40 - 33s - loss: 0.7315 - auc: 0.5054 - accuracy: 0.5026 - val_loss: 0.6228 - val_auc: 0.5569 - val_accuracy: 0.8281
Epoch 8/30
40/40 - 33s - loss: 0.7303 - auc: 0.4992 - accuracy: 0.5045 - val_loss: 

In [ ]:
prediction = model.predict(valid_generator)

In [ ]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

In [ ]:
prediction

In [ ]:
tf.__version__

In [ ]:
model.evaluate(valid_generator)